# Visualisation
Various examples of model / performance evaluation

In [2]:
import xarray as xr

ds = xr.open_zarr('data/_ZARR/canada-post_main')
ds

C:\Users\nookh\AppData\Local\Temp\ipykernel_15148\1502896160.py:3: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_zarr('data/_ZARR/canada-post_main')


<xarray.Dataset> Size: 2GB
Dimensions:     (latitude: 35, longitude: 48, time: 1462, step: 24)
Coordinates:
    number      int64 8B ...
  * latitude    (latitude) float64 280B 59.1 59.0 58.9 58.8 ... 55.9 55.8 55.7
  * step        (step) timedelta64[ns] 192B 01:00:00 ... 1 days 00:00:00
  * time        (time) datetime64[ns] 12kB 2014-12-31 2015-01-01 ... 2018-12-31
  * longitude   (longitude) float64 384B -113.3 -113.2 -113.1 ... -108.7 -108.6
    surface     float64 8B ...
    valid_time  (time, step) datetime64[ns] 281kB dask.array<chunksize=(1462, 24), meta=np.ndarray>
Data variables:
    d2m         (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
    lai_lv      (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
    lai_hv      (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
    sp          (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
    t2m         (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
    u10         (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
    tp          (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
    v10         (latitude, longitude, time, step) float32 236MB dask.array<chunksize=(35, 48, 832, 24), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-20T09:24 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
{i: i+1 for i in range(10)}

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10}

In [ ]:
from joblib import load
model = load('ADABoost_DS-0,1.joblib')

In [ ]:
import xarray as xr
from main.train.prepare_samples import prep_samples
ds = xr.open_zarr('./data/_ZARR_READY/canada-post')
X, y = prep_samples(
    ds, 
    compute_chunks = True
)

In [ ]:
import os

os.environ['SCIPY_ARRAY_API'] = '1'

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

ds = RandomUnderSampler(sampling_strategy = 0.1)
X_down, y_down = ds.fit_resample(X, y)

In [ ]:
y_pred = model.predict(X_down)

In [ ]:
import dask.array as da

full_arr = da.concatenate([X_down, y_down.reshape(-1, 1), y_pred.reshape(-1, 1)], axis=1)

In [ ]:
import dask.dataframe as df

ORIGINAL_FEATURES = ['d2m', 'lai_hv', 'lai_lv', 'mu_t2m_180', 'mu_t2m_30', 'mu_t2m_90', 'mu_tp_90', 'mu_tp_30', 'sp', 'mu_tp_180', 't2m', 'tp', 'ws10']
full_data = df.from_array(
    full_arr, columns = ORIGINAL_FEATURES + ['fire', 'predicted']
)

In [ ]:
import seaborn as sns

sns.boxenplot(
    full_data, x = 'fire', y = ''
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

x_var = 't2m'
y_var = 'mu_t2m_30'

plot_df = full_data[[x_var, y_var, 'fire']].compute()

sns.scatterplot(
    data = plot_df, x = x_var, y = y_var, hue = 'fire'
)
